In [0]:
-- =====================================================
-- DAY 5 POC - AGGREGATES & WINDOW FUNCTIONS
-- =====================================================

USE bank_management;

-- =====================================================
-- TOPIC 1: BASIC AGGREGATES
-- =====================================================

SELECT '=== Basic Aggregate Functions ===' AS example;
SELECT 
    COUNT(*) AS total_accounts,
    COUNT(last_transaction_date) AS accounts_with_activity,
    SUM(balance) AS total_deposits,
    ROUND(AVG(balance), 2) AS avg_balance,
    MIN(balance) AS min_balance,
    MAX(balance) AS max_balance
FROM accounts;

-- =====================================================
-- TOPIC 2: GROUP BY
-- =====================================================

-- Example: Accounts by type
SELECT '=== Accounts by Type (GROUP BY) ===' AS example;
SELECT 
    account_type,
    COUNT(*) AS account_count,
    ROUND(SUM(balance), 2) AS total_balance,
    ROUND(AVG(balance), 2) AS avg_balance
FROM accounts
GROUP BY account_type
ORDER BY total_balance DESC;

-- Example: Customer segments
SELECT '=== Customers by Type ===' AS example;
SELECT 
    customer_type,
    COUNT(*) AS customer_count,
    ROUND(AVG(
        (SELECT SUM(balance) FROM accounts WHERE customer_id = c.customer_id)
    ), 2) AS avg_total_balance
FROM customers c
GROUP BY customer_type
ORDER BY customer_count DESC;

-- =====================================================
-- TOPIC 3: HAVING CLAUSE
-- =====================================================

SELECT '=== Customers with Multiple Accounts (HAVING) ===' AS example;
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    COUNT(a.account_id) AS account_count,
    SUM(a.balance) AS total_balance
FROM customers c
JOIN accounts a ON c.customer_id = a.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING COUNT(a.account_id) > 1
ORDER BY account_count DESC;

-- High value accounts by type
SELECT '=== High Value Account Types (HAVING) ===' AS example;
SELECT 
    account_type,
    COUNT(*) AS account_count,
    ROUND(SUM(balance), 2) AS total_balance
FROM accounts
GROUP BY account_type
HAVING SUM(balance) > 20000
ORDER BY total_balance DESC;

-- =====================================================
-- TOPIC 4: WINDOW FUNCTIONS - ROW_NUMBER
-- =====================================================

-- Rank accounts by balance (keeps all rows)
SELECT '=== ROW_NUMBER - Rank All Accounts ===' AS example;
SELECT 
    account_id,
    account_number,
    balance,
    ROW_NUMBER() OVER (ORDER BY balance DESC) AS rank
FROM accounts
ORDER BY balance DESC
LIMIT 5;

-- Top 2 accounts per customer (PARTITION BY)
SELECT '=== Top 2 Accounts Per Customer ===' AS example;
SELECT *
FROM (
    SELECT 
        c.first_name || ' ' || c.last_name AS customer_name,
        a.account_number,
        a.balance,
        ROW_NUMBER() OVER (PARTITION BY c.customer_id ORDER BY a.balance DESC) AS rank_in_customer
    FROM customers c
    JOIN accounts a ON c.customer_id = a.customer_id
) ranked
WHERE rank_in_customer <= 2
ORDER BY customer_name, rank_in_customer;

-- =====================================================
-- TOPIC 5: RANK vs DENSE_RANK
-- =====================================================

SELECT '=== RANK vs DENSE_RANK ===' AS example;
SELECT 
    account_type,
    balance,
    RANK() OVER (ORDER BY balance DESC) AS rank_with_gaps,
    DENSE_RANK() OVER (ORDER BY balance DESC) AS rank_no_gaps,
    ROW_NUMBER() OVER (ORDER BY balance DESC) AS row_num
FROM accounts
ORDER BY balance DESC
LIMIT 8;

-- =====================================================
-- TOPIC 6: RUNNING TOTALS
-- =====================================================

SELECT '=== Running Total of Transactions ===' AS example;
SELECT 
    transaction_id,
    transaction_date,
    amount,
    SUM(amount) OVER (ORDER BY transaction_date, transaction_id) AS running_total
FROM transactions
WHERE transaction_type = 'Deposit'
ORDER BY transaction_date, transaction_id
LIMIT 10;

-- Running balance per account
SELECT '=== Running Balance Per Account ===' AS example;
SELECT 
    a.account_number,
    t.transaction_date,
    t.transaction_type,
    t.amount,
    SUM(CASE 
        WHEN t.transaction_type IN ('Deposit', 'Interest') THEN t.amount 
        ELSE -t.amount 
    END) OVER (
        PARTITION BY a.account_id 
        ORDER BY t.transaction_date, t.transaction_id
    ) AS running_balance
FROM transactions t
JOIN accounts a ON t.account_id = a.account_id
WHERE a.account_id = 2001
ORDER BY t.transaction_date, t.transaction_id
LIMIT 10;

-- =====================================================
-- TOPIC 7: LAG and LEAD
-- =====================================================

SELECT '=== Compare to Previous Transaction (LAG) ===' AS example;
SELECT 
    account_id,
    transaction_date,
    amount,
    LAG(amount, 1) OVER (PARTITION BY account_id ORDER BY transaction_date) AS previous_amount,
    amount - LAG(amount, 1) OVER (PARTITION BY account_id ORDER BY transaction_date) AS change_from_previous
FROM transactions
WHERE account_id IN (2001, 2002)
ORDER BY account_id, transaction_date
LIMIT 10;

-- =====================================================
-- INTERVIEW QUESTION 1: Top 3 Per Category
-- =====================================================

SELECT '=== Top 3 Accounts Per Type ===' AS example;
SELECT account_type, account_number, balance, rank
FROM (
    SELECT 
        account_type,
        account_number,
        balance,
        ROW_NUMBER() OVER (PARTITION BY account_type ORDER BY balance DESC) AS rank
    FROM accounts
) ranked
WHERE rank <= 3
ORDER BY account_type, rank;

-- =====================================================
-- INTERVIEW QUESTION 2: Remove Duplicates
-- =====================================================

-- Show how to identify duplicates (if existed)
SELECT '=== Deduplication Pattern ===' AS example;
SELECT 
    customer_id,
    email,
    ROW_NUMBER() OVER (PARTITION BY email ORDER BY customer_id) AS row_num
FROM customers
LIMIT 5;

-- Keep only first occurrence (row_num = 1)

-- =====================================================
-- INTERVIEW QUESTION 3: Month-over-Month Growth
-- =====================================================

SELECT '=== Monthly Transaction Trends ===' AS example;
SELECT 
    month,
    transaction_count,
    previous_month_count,
    transaction_count - previous_month_count AS growth
FROM (
    SELECT 
        DATE_TRUNC('month', transaction_date) AS month,
        COUNT(*) AS transaction_count,
        LAG(COUNT(*), 1) OVER (ORDER BY DATE_TRUNC('month', transaction_date)) AS previous_month_count
    FROM transactions
    GROUP BY DATE_TRUNC('month', transaction_date)
) monthly
ORDER BY month DESC;

-- =====================================================
-- INTERVIEW QUESTION 4: Cumulative Sum by Category
-- =====================================================

SELECT '=== Cumulative Deposits by Customer ===' AS example;
SELECT 
    c.first_name || ' ' || c.last_name AS customer_name,
    t.transaction_date,
    t.amount,
    SUM(t.amount) OVER (
        PARTITION BY c.customer_id 
        ORDER BY t.transaction_date
    ) AS cumulative_deposits
FROM customers c
JOIN accounts a ON c.customer_id = a.customer_id
JOIN transactions t ON a.account_id = t.account_id
WHERE t.transaction_type = 'Deposit' AND c.customer_id IN (1001, 1002)
ORDER BY c.customer_id, t.transaction_date
LIMIT 10;

-- =====================================================
-- INTERVIEW QUESTION 5: Percentile Rank
-- =====================================================

SELECT '=== Account Balance Percentiles ===' AS example;
SELECT 
    account_number,
    balance,
    NTILE(4) OVER (ORDER BY balance) AS quartile,
    CASE 
        WHEN NTILE(4) OVER (ORDER BY balance) = 4 THEN 'Top 25%'
        WHEN NTILE(4) OVER (ORDER BY balance) = 3 THEN 'Top 50%'
        WHEN NTILE(4) OVER (ORDER BY balance) = 2 THEN 'Top 75%'
        ELSE 'Bottom 25%'
    END AS percentile_group
FROM accounts
ORDER BY balance DESC
LIMIT 10;

-- =====================================================
-- COMPARISON: GROUP BY vs WINDOW FUNCTION
-- =====================================================

SELECT '=== GROUP BY (Collapses Rows) ===' AS example;
SELECT 
    customer_type,
    COUNT(*) AS customer_count,
    ROUND(AVG(
        (SELECT SUM(balance) FROM accounts WHERE customer_id = c.customer_id)
    ), 2) AS avg_balance
FROM customers c
GROUP BY customer_type;

SELECT '=== WINDOW FUNCTION (Keeps All Rows) ===' AS example;
SELECT 
    customer_id,
    first_name || ' ' || last_name AS customer_name,
    customer_type,
    COUNT(*) OVER (PARTITION BY customer_type) AS customers_in_type,
    ROUND(AVG(
        (SELECT SUM(balance) FROM accounts WHERE customer_id = c.customer_id)
    ) OVER (PARTITION BY customer_type), 2) AS avg_balance_in_type
FROM customers c
LIMIT 5;

-- =====================================================
-- SUMMARY
-- =====================================================

SELECT '=== DAY 5 CONCEPTS SUMMARY ===' AS summary;
SELECT 
    'COUNT/SUM/AVG/MIN/MAX' AS concept,
    'Basic aggregate functions' AS description,
    'Ignore NULLs except COUNT(*)' AS key_point
UNION ALL
SELECT 'GROUP BY', 'Collapse rows into summary', 'Must include all non-agg columns'
UNION ALL
SELECT 'HAVING', 'Filter aggregated results', 'Use after GROUP BY, not WHERE'
UNION ALL
SELECT 'ROW_NUMBER()', 'Sequential unique numbers', 'Good for deduplication'
UNION ALL
SELECT 'RANK/DENSE_RANK', 'Ranking with ties', 'RANK has gaps, DENSE_RANK no gaps'
UNION ALL
SELECT 'PARTITION BY', 'Group for window functions', 'Like GROUP BY but keeps rows'
UNION ALL
SELECT 'Running Total', 'SUM() OVER (ORDER BY)', 'Cumulative calculations'
UNION ALL
SELECT 'LAG/LEAD', 'Access previous/next row', 'Compare to previous periods';

-- =====================================================
-- END OF DAY 5 POC
-- =====================================================